In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

In [3]:
ur_files = ddf.read_csv('county_shp.csv', dtype = {'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'NAME': str, 'NAMELSAD': str})
census = ur_files.compute()
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33...."
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999..."
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999..."
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71..."
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4..."


In [4]:
census.shape

(85230, 14)

In [5]:
from shapely import wkt

In [6]:
def findCentroid(g):
    p1 = wkt.loads(g)
    return p1.centroid.wkt

In [7]:
census['Centroid'] = census['geometry'].apply(findCentroid)

In [8]:
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Centroid
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33....",POINT (-82.42777455266631 33.02009497835021)
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999...",POINT (-80.9680558426884 35.45990205837115)
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999...",POINT (-71.03573713399868 42.37754679162973)
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71...",POINT (-87.71008151357118 41.91187356531011)
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4...",POINT (-122.9834298654585 44.96158272247879)


## upload temp count data - 2010

In [9]:
years = ['1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
         '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
         '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990']

In [10]:
frames = []
for year in years:
    ur_files = ddf.read_csv('max_temp_county_level/temp_max_county_level_' + year + '.csv', dtype={'fips': 'object'})
    temp = ur_files.compute()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp.head()
    frames.append(temp)

In [11]:
temp = pd.concat(frames)
temp.head()

,fips,YEAR,TEMP,MIN,MAX
0,01001,1961.0,85.888235,78.411765,95.064706
1,01003,1961.0,83.797674,79.460465,94.067442
2,01005,1961.0,84.700000,75.544444,95.500000
3,01007,1961.0,87.400000,82.950000,96.100000
4,01009,1961.0,83.518750,75.843750,96.100000


In [12]:
temp['YEAR'].unique()

array([1961., 1962., 1963., 1964., 1965., 1966., 1967., 1968., 1969.,
       1970., 1971., 1972., 1973., 1974., 1975., 1976., 1977., 1978.,
       1979., 1980., 1981., 1982., 1983., 1984., 1985., 1986., 1987.,
       1988., 1989., 1990.])

In [13]:
temp_01001 = temp[temp['fips'] == '01001']
temp_01001

,fips,YEAR,TEMP,MIN,MAX
0,01001,1961.0,85.888235,78.411765,95.064706
0,01001,1962.0,88.705882,80.188235,100.405882
0,01001,1963.0,88.029412,80.294118,98.952941
0,01001,1964.0,86.441176,76.541176,98.347059
0,01001,1965.0,84.676471,77.388235,94.435294
0,01001,1966.0,89.100000,78.935294,104.935294
0,01001,1967.0,83.705882,76.723529,95.917647
0,01001,1968.0,85.758824,76.852941,100.388235
0,01001,1969.0,90.611765,79.823529,103.911765
0,01001,1970.0,88.082353,77.694118,99.088235


In [14]:
temp_average = temp.groupby('fips').mean()
temp_average = temp_average.reset_index()
temp_average.head()

,fips,YEAR,TEMP,MIN,MAX
0,01001,1975.5,88.086863,78.430588,99.182549
1,01003,1975.5,86.279225,80.531628,94.950310
2,01005,1975.5,85.219630,75.624444,97.295556
3,01007,1975.5,86.538333,77.821250,97.962917
4,01009,1975.5,83.823333,74.926042,96.336875


In [15]:
temp_average_1975 = temp_average[temp_average['YEAR'] == 1975.5]

In [16]:
temp_average_1975.shape

(2839, 5)

In [18]:
temp_average_1975 = temp_average_1975.rename(columns = {'TEMP': 'average_temp', 'MIN': 'average_min_temp', 'MAX': 'average_max_temp'})
temp_average_1975.head()

,fips,YEAR,average_temp,average_min_temp,average_max_temp
0,01001,1975.5,88.086863,78.430588,99.182549
1,01003,1975.5,86.279225,80.531628,94.950310
2,01005,1975.5,85.219630,75.624444,97.295556
3,01007,1975.5,86.538333,77.821250,97.962917
4,01009,1975.5,83.823333,74.926042,96.336875


In [19]:
temp_average_1975.to_csv('average_max_temp_6190.csv')